In [1]:
import os
import random
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Concatenate, Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import preprocess_input

# Load your CSV file containing pairs of similar images
csv_path = "train.csv"
data = pd.read_csv(csv_path)

# Path to your 'left' and 'right' image folders
left_folder = 'train/left/'
right_folder = 'train/right/'

# Load a pre-trained ResNet model (you can choose a different version if needed)
input_shape = (200, 245, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze all layers except the last one
for layer in base_model.layers[:-1]:
    layer.trainable = False
# Function to generate triplets
def generate_triplets(data, left_folder, right_folder):
    triplets = []
    
    for index, row in data.iterrows():
        left_image = row['left'] + '.jpg'
        right_image = row['right'] + '.jpg'
        
        # Randomly select a dissimilar image from the 'right' folder
        dissimilar_image = random.choice(os.listdir(right_folder))
        
        triplets.append((left_image, right_image, dissimilar_image))
        # Display the first triplet
        if len(triplets) == 1:
            print("First generated triplet:", triplets)
        
    return triplets

# Generate triplets and split into training and validation sets
triplets = generate_triplets(data, left_folder, right_folder)
triplets_train, triplets_val = train_test_split(triplets, test_size=0.2, random_state=42)


# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(200, 245))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Prepare data for training and validation
# Prepare data for training and validation
# Prepare data for training and validation
def prepare_data(triplets, left_folder, right_folder):
    num_triplets = len(triplets)
    X = np.zeros((num_triplets, 200, 245, 3))
    y = np.zeros((num_triplets,))

    for i, triplet in enumerate(triplets):
        left_img_path, right_img_path, dissimilar_img_path = triplet
        X[i] = load_and_preprocess_image(os.path.join(left_folder, left_img_path))
        y[i] = 1 if left_img_path == right_img_path else 0

    return X, y




# Prepare the data
X_train, y_train = prepare_data(triplets_train, left_folder, right_folder)
X_val, y_val = prepare_data(triplets_val, left_folder, right_folder)
print(len(X_train), len(y_train))
# # Train the Siamese network
# siamese_network.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# # Save the trained Siamese model
# siamese_network.save('siamese_model.h5')

ImportError: cannot import name 'load_img' from 'keras.preprocessing.image' (/Users/bwidjaja/anaconda3/envs/CV/lib/python3.9/site-packages/keras/preprocessing/image.py)